In [10]:
import numpy as np
import time

# Adversarial Search for multi-player games

## Node
The Node class greates the graph node. It has the following values
1. State 
2. children - List of children
3. score - value of node
4. depth - depth of node in game tree
5. maximiser - boolean value, whether node is a maximiser node or minimiser
6. player - value of player, whether x or o
7. best_child - best child of current node (Needed for finding path)

It makes use of the following built in functions: 
1. \_\_hash\_\_ : This provides the hash value for every node, which is required for the hashset

In [11]:
class Node():
    
    def __init__(self, state, depth, maximiser, player, parent=None,):
        self.state = state
        self.children = list()
        self.score = 0
        self.depth = depth
        self.maximiser = maximiser
        self.player = player
        self.best_child = None
        
        if parent is not None:
            parent.children.append(self)
        
    def __hash__(self):
        
        return hash(str(self.state))
    
    

## Environment

The environment is what the agent plays in. It has the following entities:
1. goal_state : The goal state of the environment
2. start_state : The start state generated at the depth

It has the following functions: 
1. get_moves : returns all possible next moves from a given configuration
2. check_terminal : checks whether all cells have been filled or not
3. evalauate : returns the value for every state. +1 for win, -1 for lose and 0 for draw
4. generate_start_state : Given goal state and depth d, performs d moves to generate a start state 

In [12]:
class Environment():
    
    def __init__(self, start_state=None):
        
        if start_state is None:
            self.start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
        else:
            self.start_state = start_state
    def get_moves(self, state, player):
        
        new_states = []
        spaces = []
        for i in range(3):
            for j in range(3):
                if state[i][j]=='.':
                    new_state = state.copy()
                    new_state[i,j] = player
                    new_states.append(new_state)
        
        return new_states
    
    def check_terminal(self, state):
        for i in range(3):
            for j in range(3):
                if state[i][j]=='.':
                    return False
        
        return True
    
    def evaluate(self, state):
        
        for val in range(3):
            if state[val,0] == state[val,1] == state[val,2]!='.':
                if state[val, 0]=='x':
                    return 1
                else:
                    return -1
            
            if state[0,val] == state[1,val] == state[2,val]!='.':
                if state[0,val]=='x':
                    return 1
                else:
                    return -1
        
        if state[0,0] == state[1,1] == state[2,2]!='.':
            if state[0,0]=='x':
                return 1
            else:
                return -1
        
        if state[0,2] == state[1,1] == state[2,0]!='.':
            if state[0,2]=='x':
                return 1
            else:
                return -1
        
        return 0

    def get_start_state(self):
        return self.start_state


## Agent
The agent is the player who plays the game against the environment to win. It has the following entities:
1. env : Stores the environment
2. start_state : Stores the start state
3. root_node : Stores the root node of the tree
4. maximiser : If root is a maximiser or minimiser
5. player : If root player is x or o
6. explored_nodes : Counts nodes explored in the tree

The agent has the following functions: 
1. minimax() : Is the recursive function to evaluate the tree 
1. run(): Is the function that explores the environment and finds the goal node.
2. print_nodes(): To print the path from the start node to goal node

In [13]:
class Agent():
    
    def __init__(self, env, maximiser):
        
        self.env = env
        self.start_state = env.get_start_state()
        self.root_node = None
        self.neginf = -10**18
        self.posinf = 10**18
        self.maximiser = maximiser
        self.player = 'x' if maximiser else 'o';
        self.explored_nodes = 0
    
    def minimax(self, node):
        
        self.explored_nodes+=1
        score = self.env.evaluate(node.state)
        if score!=0:
            node.score = score
            return node
        
        if self.env.check_terminal(node.state):
            node.score = 0
            return node
        
        if node.maximiser:
            
            best_score = self.neginf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='o', parent=node)
                
                child= self.minimax(child)
                node.children.append(child)
                
                if best_score<child.score or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
            
            node.depth = best_depth
            node.score = best_score
            
            return node
        
        else:
            best_score = self.posinf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='x', parent=node)
                
                child = self.minimax(child)
                node.children.append(child)
                
                if best_score>child.score  or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
            
            node.depth = best_depth
            node.score = best_score
            
            return node

    def run(self):
        
        self.root_node = Node(state=self.start_state, depth=0, maximiser=self.maximiser,
                             player=self.player, parent=None)
        
        self.root_node = self.minimax(node = self.root_node)
        
    def print_nodes(self):
        
        node = self.root_node
        
        while node is not None:
            print(node.state)
            node = node.best_child

In [14]:
t = 0
for i in range(10):
    start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
    env = Environment(start_state = start_state)
    agent = Agent(env, maximiser=True)
    start_time = time.time()
    agent.run()
    end_time = time.time()
    t += end_time-start_time
print("Average time required:", t/10)

Average time required: 11.156453704833984


In [16]:
agent.print_nodes()

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' 'x']]


In [17]:
agent.explored_nodes

549946

## AlphaBetaAgent
The agent is the player who plays the game against the environment to win. It has the following entities:
1. env : Stores the environment
2. start_state : Stores the start state
3. root_node : Stores the root node of the tree
4. maximiser : If root is a maximiser or minimiser
5. player : If root player is x or o
6. explored_nodes : Counts nodes explored in the tree

The agent has the following functions: 
1. minimax() : Is the recursive function to evaluate the tree, passes extra parameters alpha and beta
1. run(): Is the function that explores the environment and finds the goal node.
2. print_nodes(): To print the path from the start node to goal node

In [6]:
class AlphaBetaAgent():
    
    def __init__(self, env, maximiser):
        
        self.env = env
        self.start_state = env.get_start_state()
        self.root_node = None
        self.neginf = -10**18
        self.posinf = 10**18
        self.maximiser = maximiser
        self.player = 'x' if maximiser else 'o';
        self.explored_nodes = 0
    
    
    def minimax(self, node, alpha, beta):
        
        self.explored_nodes+=1
        score = self.env.evaluate(node.state)
        if score!=0:
            node.score = score
            return node
        
        if self.env.check_terminal(node.state):
            node.score = 0
            return node
        
        if node.maximiser:
            
            best_score = self.neginf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='o', parent=node)
                
                child= self.minimax(node = child, alpha=alpha, beta=beta)
                node.children.append(child)
                
                if best_score<child.score or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
                
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
            
            node.depth = best_depth
            node.score = best_score
            
            return node
        
        else:
            best_score = self.posinf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='x', parent=node)
                
                child = self.minimax(node = child, alpha=alpha, beta=beta)
                node.children.append(child)
                
                if best_score>child.score  or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
                
                beta = min(beta, best_score)
                if beta<=alpha:
                    break
            
            node.depth = best_depth
            node.score = best_score
            
            return node

    def run(self):
        
        self.root_node = Node(state=self.start_state, depth=0, maximiser=self.maximiser,
                             player=self.player, parent=None)
        
        self.root_node = self.minimax(node = self.root_node, alpha = self.neginf, beta = self.posinf)
        
    def print_nodes(self):
        
        node = self.root_node
        
        while node is not None:
            print(node.state)
            node = node.best_child

In [9]:
start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
env = Environment(start_state = start_state)
agent = AlphaBetaAgent(env, maximiser=True)
agent.run()
agent.print_nodes()

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' 'x']]


In [7]:
import time

In [15]:
t = 0
for i in range(10):
    start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
    env = Environment(start_state = start_state)
    agent = AlphaBetaAgent(env, maximiser=True)
    start_time = time.time()
    agent.run()
    end_time = time.time()
    t += end_time-start_time
print("Average time required:", t/10)

Average time required: 0.31589887142181394


In [13]:
agent.print_nodes()

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['x' '.' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['x' 'x' 'o']
 ['.' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' '.']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' '.' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' '.']]
[['x' 'x' 'o']
 ['o' 'o' 'x']
 ['x' 'o' 'x']]


In [14]:
agent.explored_nodes

18297